In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
import re
import gensim

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CapoBizkitz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CapoBizkitz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CapoBizkitz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/Tariq60/LIAR-PLUS/master/dataset/tsv/train2.tsv', delimiter='\t', encoding='utf-8', names=
                        ["json ID", "label", "statement", "subject", "speaker", "job title", "state", "party",
                         "barely true", "false", "half true", "mostly true", "pants on fire", "los", "justification"])
train.name = 'Training Data'

test = pd.read_csv('https://raw.githubusercontent.com/Tariq60/LIAR-PLUS/master/dataset/tsv/test2.tsv', delimiter='\t', encoding='utf-8', names=
                        ["json ID", "label", "statement", "subject", "speaker", "job title", "state", "party",
                         "barely true", "false", "half true", "mostly true", "pants on fire", "los", "justification"])
test.name = 'Testing Data'

In [3]:
dataset = train.append(test, ignore_index=True)

<ipython-input-3-8cd8721b725d>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = train.append(test, ignore_index=True)


In [4]:
dataset.shape

(11509, 15)

In [5]:
null_columns=train.columns[dataset.isnull().any()]
dataset[null_columns].isnull().sum()

json ID             2
label               2
statement           2
subject             4
speaker             4
job title        3224
state            2472
party               4
barely true         4
false               4
half true           4
mostly true         4
pants on fire       4
los               121
justification      97
dtype: int64

In [6]:
dataset = dataset[dataset['json ID'].notna()]

In [7]:
print(dataset[dataset["label"].isnull()][null_columns])

Empty DataFrame
Columns: [json ID, label, statement, subject, speaker, job title, state, party, barely true, false, half true, mostly true, pants on fire, los, justification]
Index: []


In [8]:
dataset.shape

(11507, 15)

In [9]:
dataset

,json ID,label,statement,subject,speaker,job title,state,party,barely true,false,half true,mostly true,pants on fire,los,justification
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28.0,23.0,38.0,34.0,7.0,a news conference,LeMieux didn't compare Rubio and Obama on an i...
11505,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2.0,0.0,0.0,1.0,0.0,"on ABC's ""This Week""","After making his pledge, Obama said the budget..."
11506,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1.0,0.0,0.0,0.0,0.0,a commentary in The Providence Journal,Former President Clinton said government got t...
11507,3186.json,barely-true,Says an EPA permit languished under Strickland...,"environment,government-efficiency",john-kasich,"Governor of Ohio as of Jan. 10, 2011",Ohio,republican,9.0,8.0,10.0,18.0,3.0,a news conference,Points of Light has a unique mission carved ou...


In [10]:
dataset = dataset[dataset['party'].notna()]

In [11]:
dataset.shape

(11505, 15)

In [12]:
dataset[dataset["party"] == 'none']

,json ID,label,statement,subject,speaker,job title,state,party,barely true,false,half true,mostly true,pants on fire,los,justification
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
13,8616.json,mostly-true,The economy bled $24 billion due to the govern...,"economy,federal-budget,health-care",doonesbury,NaN,NaN,none,0.0,0.0,2.0,4.0,0.0,a Doonesbury strip in the Sunday comics,"Doonesbury, aka Garry Trudeau, said the govern..."
21,9867.json,mostly-true,The United States has the highest corporate ta...,"corporations,taxes",eric-bolling,"Co-host on Fox News Channel's ""The Five""",NaN,none,2.0,1.0,1.0,1.0,0.0,"a discussion on Fox News' ""The Five""","Bolling said the United States has ""the highes..."
23,2673.json,half-true,"Says Scott Walker favors cutting up to 350,000...","health-care,message-machine",greater-wisconsin-political-fund,NaN,Wisconsin,none,3.0,3.0,3.0,1.0,1.0,a campaign TV ad,"Walker changed course to a muddier position, w..."
24,7057.json,barely-true,Says Mitt Romney wants to get rid of Planned P...,"abortion,federal-budget,health-care",planned-parenthood-action-fund,Advocacy group,"Washington, D.C.",none,1.0,0.0,0.0,0.0,0.0,a radio ad,The Planned Parenthood Action Fund aid said th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11473,10547.json,true,Georgias share of money from the Federal Highw...,"state-budget,state-finances,transportation",associated-press,journalists,NaN,none,0.0,0.0,0.0,0.0,0.0,in a newspaper article,And Bader suggested that because Sensenbrenner...
11476,6045.json,pants-fire,Says Hawaii Gov. Neil Abercrombie made a late-...,obama-birth-certificate,chain-email,NaN,NaN,none,11.0,43.0,8.0,5.0,105.0,e-mail messages circulating on the Internet,"In every case, Southerland eventually fully sa..."
11483,5746.json,true,The U.S. Supreme Court has not traditionally a...,supreme-court,clarence-thomas,U.S. Supreme Court Justice,Georgia,none,0.0,0.0,0.0,0.0,0.0,a speech,Gillespie said this is the first time in more ...
11499,1005.json,pants-fire,Obama used $20 million in federal money to emm...,foreign-policy,chain-email,NaN,NaN,none,11.0,43.0,8.0,5.0,105.0,a chain e-mail,"Thomas, the third-longest tenured justice on t..."


In [13]:
dataset = dataset[dataset["party"].str.contains("none") == False]

In [14]:
dataset = dataset[dataset["subject"].str.contains("baseball,recreation,sports") == False]

In [15]:
dataset = dataset[dataset["subject"].str.contains("energy,environment") == False]

In [16]:
dataset = dataset[dataset["party"].str.contains("education-official") == False]

In [17]:
dataset.shape

(9420, 15)

In [19]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to c:\users\capobizkitz\appdata\local\temp\pip-req-build-na50rgdv
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7683 sha256=4ea37215456e33227be7635495957ec34be94cca72373284b6e2e5a98009ae4a
  Stored in directory: C:\Users\CapoBizkitz\AppData\Local\Temp\pip-ephem-wheel-cache-wan16l_r\wheels\fd\71\47\e04f208ad659a295ccb69022e14de7d20ccc6fc30a5a935f09
Successfully built preprocess-kgptalkie
  Attempting uninstall: preprocess-kgptalkie
    Found existing installation: preprocess-kgptalkie 0.1.3
    Uninstalling preprocess-kgptalkie-0.1.3:
      Successfully uninstalled preprocess-kgptalkie-0.1.3


In [21]:
dataset['new'] = dataset['statement'].map(str) + dataset['justification'].map(str)

In [22]:
import preprocess_kgptalkie as ps

dataset['new'].apply(lambda x: ps.remove_special_chars(x))

0        Says the Annies List political group supports ...
1        When did the decline of coal start It started ...
2        Hillary Clinton agrees with John McCain by vot...
4        The economic turnaround started at the end of ...
5        The Chicago Bears have had more starting quart...
                               ...                        
11504    Says his budget provides the highest state fun...
11505    Ive been here almost every dayAfter making his...
11506    In the early 1980s Sen Edward Kennedy secretly...
11507    Says an EPA permit languished under Strickland...
11508    Says the governor is going around the state ta...
Name: new, Length: 9420, dtype: object

In [23]:
# Remove stopwords and remove words with 2 or less characters using gensim
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [32]:
def Lammetize(text):
    word_list = nltk.word_tokenize(text)
    lammetized_output = ' '.join([lemmatizer.lemmatize(w, 'v') for w in word_list])

    return lammetized_output

In [33]:
ps = PorterStemmer()

def Stemming(text):
    
    word_list = nltk.word_tokenize(text)
    stemmed_output = ' '.join([ps.stem(w) for w in word_list])

    return stemmed_output

In [24]:
dataset['new'] = dataset['new'].apply(preprocess)

In [26]:
dataset['new'] = dataset['new'].apply(lambda x: " ".join(x))

In [28]:
dataset['new'].str.strip()

0        says annies list political group supports trim...
1        decline coal start started natural took starte...
2        hillary clinton agrees john mccain voting geor...
4        economic turnaround started term crist said ec...
5        chicago bears starting quarterbacks years tota...
                               ...                        
11504    says budget provides highest state funding lev...
11505    making pledge obama said budget soon propose l...
11506    early edward kennedy secretly offered help sov...
11507    says permit languished strickland director day...
11508    says governor going state talking fund income ...
Name: new, Length: 9420, dtype: object

In [34]:
dataset['new'] = dataset['new'].apply(Lammetize)
dataset['new'] = dataset['new'].apply(Stemming)

In [35]:
dataset['new']

0        say anni list polit group support trimest abor...
1        declin coal start start natur take start begin...
2        hillari clinton agre john mccain vote georg bu...
4        econom turnaround start term crist say econom ...
5        chicago bear start quarterback year total numb...
                               ...                        
11504    say budget provid highest state fund level his...
11505    make pledg obama say budget soon propos look a...
11506    earli edward kennedi secretli offer help sovie...
11507    say permit languish strickland director day po...
11508    say governor go state talk fund incom benefit ...
Name: new, Length: 9420, dtype: object

In [36]:
dataset = dataset.rename(columns = {'json ID': 'articleId', 'new': 'cleaned_statement'}, inplace = False)

In [37]:
dataset['articleId'] = dataset['articleId'].str.split('.').str[0]

In [38]:
new_dataset = dataset[['articleId', 'cleaned_statement', 'label']]

In [41]:
new_dataset

,articleId,cleaned_statement,label
0,2635,say anni list polit group support trimest abor...,false
1,10540,declin coal start start natur take start begin...,half-true
2,324,hillari clinton agre john mccain vote georg bu...,mostly-true
4,9028,econom turnaround start term crist say econom ...,half-true
5,12465,chicago bear start quarterback year total numb...,true
...,...,...,...
11504,7334,say budget provid highest state fund level his...,half-true
11505,9788,make pledg obama say budget soon propos look a...,barely-true
11506,10710,earli edward kennedi secretli offer help sovie...,barely-true
11507,3186,say permit languish strickland director day po...,barely-true


In [42]:
new_dataset.to_csv(r'political.csv', index = False)